```
[[1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
]
 
 [
 [X X _ _ _ _]
 [X X X X _ _]
 [X X X X _ _]
 [X X X X X _]
 [X X X X X X]
 [X X X X X X]
 [X X X X X X]
 [X X X X X X]
 [_ X X X X _]
 [_ _ _ _ X _]
 ]
 
 ```

In [1]:
from __future__ import print_function
import sys
from ortools.sat.python import cp_model

In [2]:
class VarArraySolutionPrinterWithLimit(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0
        self.__solution_limit = limit

    def on_solution_callback(self):
        self.__solution_count += 1
        for row in self.__variables:
            for v in row:
                print('%i' % (self.Value(v)), end=' ')
            print()
        print()
        if self.__solution_count >= self.__solution_limit:
            print('Recherche stoppée après %i solutions' % self.__solution_limit)
            self.StopSearch()
        
        
    def solution_count(self):
        return self.__solution_count


In [73]:
def main():
    
    representation2D = [[1., 1., 0., 0., 0., 0.],
 [1., 1., 1., 1., 0., 0.],
 [1., 1., 1., 1., 0., 0.],
 [1., 1., 1., 1., 1., 0.],
 [1., 1., 1., 1., 1., 1.],
 [1., 1., 1., 1., 1., 1.],
 [1., 1., 1., 1., 1., 1.],
 [1., 1., 1., 1., 1., 1.],
 [0., 1., 1., 1., 1., 0.],
 [0., 0., 0., 0., 1., 0.],
]
    
    size_x = len(representation2D[0])
    size_y = len(representation2D)
        
    model = cp_model.CpModel()
    
    # Liste pour accéder d'avantage aux différentes zones géographiques
    northern_part = []
    southern_part = []
    western_part = []
    eastern_part = []
    
    empty_part = []
    
    ground = []
    for i in range(size_y): # Nb Lignes
        ground.append([])
        for j in range(size_x): # Nb colonnes
            if representation2D[i][j] == 0:
                empty_var = model.NewIntVar(0, 0,'x(%i,%i)' % (i,j))
                ground[i].append(empty_var)
                empty_part.append(empty_var.Name())
            else:
                ground[i].append(model.NewIntVar(1, 2,'x(%i,%i)' % (i,j)))
            # Partie Ouest
            if j < size_x // 2:
                western_part.append(ground[i][j])
            elif j >= size_x // 2: # Partie Est
                eastern_part.append(ground[i][j])

        # Partie Nord
        if i < size_y // 2:     
            northern_part += [ground[i][j] for j in range(size_x)]
        elif i >= size_y // 2: # Partie Sud
            southern_part += [ground[i][j] for j in range(size_x)]
   
    """
    for i in range(size_y):
        for j in range(size_x):
            # Check que l'on soit dans la grille
            
            if i > 0 and i < size_y - 1  and j > 0 and j < size_x - 1:
               
                # Si var courante n'est pas un 0
                if ground[i][j].Name() not in empty_part:
                    l = [ground[i-1][j].Name(), ground[i-1][j-1].Name(),
                         ground[i-1][j+1].Name(), ground[i][j-1].Name(),
                         ground[i][j+1].Name(), ground[i+1][j-1].Name(),
                         ground[i+1][j].Name(), ground[i+1][j+1].Name()]
                    
                    if not any(elem in l for elem in empty_part):
                        # On a dessiné ici notre carré
                        #import pdb
                       # pdb.set_trace()
                        model.Add(ground[i][j] == 2)
                        model.Add(ground[i-1][j] == 2)
                        model.Add(ground[i-1][j-1] == 2)
                        model.Add(ground[i-1][j+1] == 2)
                        model.Add(ground[i][j-1] == 2)
                        model.Add(ground[i][j+1] == 2)
                        model.Add(ground[i+1][j-1] == 2)
                        model.Add(ground[i+1][j] == 2)
                        model.Add(ground[i+1][j+1] == 2)
                #if i == 2 and j == 1:
                          #  model.Add(ground[i+3][j] == 1)
                            
     """

    # On va mettre ici des contraintes (le plus important actuellement)            
    solver = cp_model.CpSolver()

    solution_printer = VarArraySolutionPrinterWithLimit(ground, 10)
    status = solver.SearchForAllSolutions(model, solution_printer)
    print('Status  = %s' % solver.StatusName(status))
    print('Nombre de solutions trouvées: %i' % solution_printer.solution_count())
    #assert solution_printer.solution_count() == 10

In [74]:
main()

TypeError: 'IntVar' object is not iterable